# 03 - Medium Roaster 

This notebook implements our balanced CV critique model.

## Characteristics
- Professional but honest tone & direct feedback without sugarcoating
- Points out obvious issues clearly while maintains professionalism
- Using medium temperature (0.6-0.8) for more varied and candid responses

---

In [1]:
import pandas as pd
import json
from pathlib import Path
from datetime import datetime
import sys
sys.path.append('..')
import google.generativeai as genai

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


/Users/hannokuegler/Library/CloudStorage/OneDrive-WUWien/SBWL/Data Science/4_LLM/roast_my_cv/roast_my_cv/.venv1/lib/python3.9/site-packages/google/api_core/_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.6) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/hannokuegler/Library/CloudStorage/OneDrive-WUWien/SBWL/Data Science/4_LLM/roast_my_cv/roast_my_cv/.venv1/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Setup

In [2]:
# Load API key from config.py
import sys
sys.path.append('..')
from config import GEMINI_API_KEY
genai.configure(api_key=GEMINI_API_KEY)
print("API key loaded from config.py")

API key loaded from config.py


## Load Data and Helper Functions

In [3]:
# Load dataset
df = pd.read_csv('../data/resume_data.csv')

# Load test CV indices
with open('../data/test_cv_indices.json', 'r') as f:
    test_data = json.load(f)
    test_cv_indices = test_data['indices']

print(f"Loaded {len(df)} resumes")
print(f"Test CVs: {test_cv_indices}")

Loaded 9544 resumes
Test CVs: [0, 1]


In [4]:
# CV formatting function
def format_cv_for_llm(resume_row):
    """
    Format a resume row into a readable text for LLM processing.
    """
    cv_text = []
    
    if pd.notna(resume_row.get('career_objective')):
        cv_text.append(f"CAREER OBJECTIVE:\n{resume_row['career_objective']}")
    
    if pd.notna(resume_row.get('skills')):
        cv_text.append(f"\nSKILLS:\n{resume_row['skills']}")
    
    education_parts = []
    if pd.notna(resume_row.get('educational_institution_name')):
        education_parts.append(f"Institution: {resume_row['educational_institution_name']}")
    if pd.notna(resume_row.get('degree_names')):
        education_parts.append(f"Degree: {resume_row['degree_names']}")
    if pd.notna(resume_row.get('major_field_of_studies')):
        education_parts.append(f"Major: {resume_row['major_field_of_studies']}")
    if pd.notna(resume_row.get('passing_years')):
        education_parts.append(f"Year: {resume_row['passing_years']}")
    
    if education_parts:
        cv_text.append(f"\nEDUCATION:\n" + "\n".join(education_parts))
    
    work_parts = []
    if pd.notna(resume_row.get('professional_company_names')):
        work_parts.append(f"Company: {resume_row['professional_company_names']}")
    if pd.notna(resume_row.get('positions')):
        work_parts.append(f"Position: {resume_row['positions']}")
    if pd.notna(resume_row.get('start_dates')):
        work_parts.append(f"Period: {resume_row['start_dates']}")
        if pd.notna(resume_row.get('end_dates')):
            work_parts.append(f" to {resume_row['end_dates']}")
    if pd.notna(resume_row.get('responsibilities')):
        work_parts.append(f"Responsibilities:\n{resume_row['responsibilities']}")
    
    if work_parts:
        cv_text.append(f"\nWORK EXPERIENCE:\n" + "\n".join(work_parts))
    
    if pd.notna(resume_row.get('languages')):
        cv_text.append(f"\nLANGUAGES:\n{resume_row['languages']}")
    
    if pd.notna(resume_row.get('certification_skills')):
        cv_text.append(f"\nCERTIFICATIONS:\n{resume_row['certification_skills']}")
    
    return "\n".join(cv_text)

## Medium Roaster Prompt Design

In [5]:
MEDIUM_SYSTEM_PROMPT = """You are an experienced hiring manager who provides direct, honest CV feedback.

Your approach:
1. Be direct and honest - no sugarcoating
2. Point out obvious flaws and red flags
3. Call out generic buzzwords and filler content
4. Be professional but don't hold back the truth
5. Focus on what actually matters to employers

Keep your feedback:
- Brutally honest but professional
- Direct about weaknesses
- Critical of vague or generic content
- Focused on real-world hiring standards

Structure your response:
 FIRST IMPRESSION: What stands out (good or bad)
 MAJOR ISSUES: Glaring problems that need fixing
 CONCERNS: Things that raise questions
 WHAT WORKS: Brief acknowledgment of strengths
 BOTTOM LINE: Final verdict and priority fixes
"""

def create_medium_prompt(cv_text):
    """Create a medium roasting prompt."""
    return f"""Review this CV with honest, direct feedback. Don't hold back on pointing out issues:

{cv_text}

Provide your honest assessment following the structure in the system prompt."""

## Temperature Tuning Experiments

Let's test different temperatures for more varied and candid responses.

In [6]:
def roast_cv(cv_text, temperature=0.7, model_name="gemini-2.0-flash"):
    """
    Generate CV critique using Gemini.
    
    Args:
        cv_text: Formatted CV text
        temperature: Controls randomness (0.0-1.0)
        model_name: Gemini model to use
    
    Returns:
        str: Generated critique
    """
    model = genai.GenerativeModel(
        model_name=model_name,
        generation_config=genai.GenerationConfig(
            temperature=temperature,
            top_p=0.95,
            top_k=40,
            max_output_tokens=1024,
        )
    )
    
    full_prompt = f"{MEDIUM_SYSTEM_PROMPT}\n\n{create_medium_prompt(cv_text)}"
    
    response = model.generate_content(full_prompt)
    return response.text

# Test with first CV
test_cv = format_cv_for_llm(df.iloc[test_cv_indices[0]])
print("Test CV:")
print("="*80)
print(test_cv[:500] + "...")
print("="*80)

Test CV:
CAREER OBJECTIVE:
Big data analytics working and database warehouse manager with robust experience in handling all kinds of data. I have also used multiple cloud infrastructure services and am well acquainted with them. Currently in search of role that offers more of development.

SKILLS:
['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapreduce', 'Spark', 'Java', 'Machine Learning', 'Cloud', 'Hdfs', 'YARN', 'Core Java', 'Data Science', 'C++', 'Data Structures', 'DBMS', 'RDBMS', 'Informatica', 'Talend...


### Experiment 1: Medium Temperature (0.6)
More varied but still controlled

In [7]:
print(" Temperature: 0.6 (Controlled Variety)")
print("="*80)
result_temp_06 = roast_cv(test_cv, temperature=0.6)
print(result_temp_06)
print("\n" + "="*80)

 Temperature: 0.6 (Controlled Variety)
Okay, here's my brutally honest assessment of this CV:

**FIRST IMPRESSION:** This CV screams "generic and underdeveloped." It's a list of keywords followed by vague responsibilities. It lacks substance and doesn't showcase any real achievements. The formatting is also questionable, especially the "to ['Till Date']" under work experience.

**MAJOR ISSUES:**

*   **Career Objective is terrible:** It's a rambling, buzzword-filled sentence that says nothing concrete. "Robust experience in handling all kinds of data" is meaningless. What kind of data? What kind of experience? And the shift to wanting "more of development" is confusing. Is this a data analyst or a developer? This needs a complete rewrite or, better yet, deletion. Objectives are outdated.
*   **Skills Section is just a keyword dump:** Listing skills without demonstrating them is useless. Anyone can copy and paste a list of technologies. Where's the evidence of proficiency? What projects

### Experiment 2: Medium-High Temperature (0.7)
Good balance of creativity and directness

In [8]:
print(" Temperature: 0.7 (Balanced)")
print("="*80)
result_temp_07 = roast_cv(test_cv, temperature=0.7)
print(result_temp_07)
print("\n" + "="*80)

 Temperature: 0.7 (Balanced)
Okay, here's my brutally honest assessment of this CV:

**FIRST IMPRESSION:** This looks like a very junior profile trying to sound more experienced than they are. The career objective is weak, the skills section is a massive keyword dump, and the work experience is shockingly vague.

**MAJOR ISSUES:**

*   **Career Objective is Terrible:** "Big data analytics working and database warehouse manager" is not a coherent objective. It reads like you threw a bunch of buzzwords together. And the last sentence "Currently in search of role that offers more of development" is poorly written and tells me you're already dissatisfied with your current role. This needs a complete rewrite, focusing on a specific target role and highlighting what *you* bring to the table, not what you *want*.
*   **Skills Section is a Keyword Vomit:** Listing every buzzword under the sun doesn't make you an expert. It makes you look like you're trying to game the system. You need to *demo

### Experiment 3: High Temperature (0.8)
More creative and varied critiques

In [9]:
print(" Temperature: 0.8 (More Creative)")
print("="*80)
result_temp_08 = roast_cv(test_cv, temperature=0.8)
print(result_temp_08)
print("\n" + "="*80)

 Temperature: 0.8 (More Creative)
Okay, let's dissect this CV. Prepare for some tough love.

**FIRST IMPRESSION:** This reads like a template someone filled in halfway and then gave up. It's incredibly generic, lacks substance, and immediately raises red flags.

**MAJOR ISSUES:**

*   **Career Objective:** This is terrible. "Big data analytics working and database warehouse manager..." is grammatically awkward and doesn't say *anything* about what you bring to the table or what kind of development role you're seeking. It reads like you just mashed together keywords. The mention of cloud infrastructure is vague and meaningless without context. This needs a complete rewrite to focus on your target role and quantifiable achievements.
*   **Skills Section:** This is just a laundry list of buzzwords. Listing technologies without demonstrating proficiency or context is useless. Anyone can copy and paste a list of skills; hiring managers want to see *how* you've used them to solve problems an

## Select Optimal Temperature

Based on reading through the experiments above, we manually select the temperature that provides a good balance of:
- Direct, honest feedback with critique points in candid tone

**Recommended: 0.7 for medium roasting**


In [10]:
# Set optimal temperature
OPTIMAL_TEMPERATURE = 0.7

print(f" Selected optimal temperature: {OPTIMAL_TEMPERATURE}")

 Selected optimal temperature: 0.7


## Summary

This notebook demonstrated:
1.  Direct CV critique prompt design
2.  Temperature tuning experiments (0.6, 0.7, 0.8)
3.  Selection of optimal temperature
4.  Generation of critiques
5.  Saving results for comparison